# 에스티메이터
## 1. 사전 제작된 에스티메이터
### 참고사이트 : https://www.tensorflow.org/tutorials/estimator/premade?hl=ko

In [2]:
import tensorflow as tf

import pandas as pd

In [6]:
# dataset

# Iris 꽃을 꽃받침과 꽃잎의 크기에 따라 3가지 종으로 분류하는 모델을 구축하고 테스트
# Iris dataset에는 4가지 Feature(꽃받침의 길이,너비 / 꽃잎의 길이,너비)와 Label(답)이 있음.

# 위 정보를 바탕으로 데이터 구문 분석에 도움이 될만한 몇 가지 상수 정의
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [7]:
# Keras 및 Pandas를 사용해 Iris dataset 다운로드
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

# 교육 및 테스트를 위해 별도의 dataset 유지
train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

In [10]:
train.head() # 처음 5개 데이터 확인을 통해 값들이 모두 정상적인지 확인

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [11]:
# 모델 예측을 위해 학습 할 label 분할
train_y = train.pop('Species')
test_y = test.pop('Species')

# The label column has now been removed from the features.
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


### Estimators를 사용한 프로그래밍 개요
 1) 하나 이상의 입력 기능 만들기    
 2) 모델의 feature 열 정의   
 3) feature 열과 다양한 파라미터를 지정해 Estimator를 인스턴스화   
 4) 적절한 입력 함수를 데이터 소스로 전달해 Estimator object에서 하나 이상의 메소드를 호출


In [12]:
# 1) 입력 기능 생성 => 0,1번을 가지고 만든 간단한 예제
# 학습, 평가, 예측을 위한 데이터 제공을 위해서 입력 기능 작성
# features = python dictionary 타입 { feature이름 : 해당 feature의 모든 값 }
# label = 모든 예제의 label 값을 포함하는 배열
def input_evaluation_set():
    features = {'SepalLength': np.array([6.4, 5.0]),
                'SepalWidth':  np.array([2.8, 2.3]),
                'PetalLength': np.array([5.6, 3.3]),
                'PetalWidth':  np.array([2.2, 1.0])}
    labels = np.array([2, 1])
    return features, labels

In [13]:
# 이 예제에서 작업을 단순하게 유지하기 위해 pandas로 데이터를 로드하고 이 안에서 입력 파이프라인 작성
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [14]:
# 2) 모델의 feature column 정의
# estimator 모델을 빌드 할 때 모델에서 사용 할 각 기능을 설명하는 기능 열 목록(옵션 다양)
# Iris dataset의 경우 4 개의 행은 숫자 값이므로, Estimator 모델에 4 가지 기능 각각을 32 비트 부동 소수점 값으로 표시하도록 기능 열 목록을 작성
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
    
# feature column은 더 정교하게 나눌 수 있음(https://www.tensorflow.org/tutorials/structured_data/feature_columns?hl=ko)
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [15]:
# 3) feature 열과 다양한 파라미터를 지정해 Estimator를 인스턴스화
# tf.estimator.DNNClassifier 사용(30개 + 10 개의 hidden nodes를 포함하는 2개의 hidden layers)
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp0zyb7ayi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
# 4) 훈련, 평가, 예측
# 위에서(3번) Estimator object를 생성했기 때문에 훈련, 평가, 예측이 가능해짐

# 모델 훈련(학습)
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
IN

In [17]:
# 훈련된 모델 평가
# 성능에 대한 통계를 얻기 위해 test_data를 통해 학습된 모델의 정확성 평가
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result)) # 93.3%의 정확도!

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-21T15:20:49Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp0zyb7ayi/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.43787s
INFO:tensorflow:Finished evaluation at 2020-04-21-15:20:50
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.93333334, averag

In [30]:
# 훈련된 모델에서 예측하기
# 평가 결과 우수한 훈련 모델이 생성됨(93.3%의 정확도)
# 훈련되지 않은 모델을 사용해 Iris 꽃의 종을 예측 할 수 있음
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(input_fn=lambda: input_fn(predict_x))
print(predictions)

<generator object Estimator.predict at 0x7f2967754eb8>


In [31]:
# 위의 예측 데이터 3가지가 무슨 종류인지 결과 내놓음
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp0zyb7ayi/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (87.6%), expected "Setosa"
Prediction is "Versicolor" (56.0%), expected "Versicolor"
Prediction is "Virginica" (59.5%), expected "Virginica"
